In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1709833511192_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%bash
# This is a Bash cell
wget -O - https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz | aws s3 cp - s3://store-raw-data-fs/city_market_tracker.tsv000.gz

--2024-03-07 19:36:14--  https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz
Resolving redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)... 52.218.221.153, 3.5.76.141, 3.5.76.136, ...
Connecting to redfin-public-data.s3.us-west-2.amazonaws.com (redfin-public-data.s3.us-west-2.amazonaws.com)|52.218.221.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1223195819 (1.1G) [application/x-www-form-urlencoded]
Saving to: ‘STDOUT’

     0K .......... .......... .......... .......... ..........  0%  848K 23m29s
    50K .......... .......... .......... .......... ..........  0%  850K 23m27s
   100K .......... .......... .......... .......... ..........  0%  124M 15m41s
   150K .......... .......... .......... .......... ..........  0%  837K 17m43s
   200K .......... .......... .......... .......... ..........  0%  103M 14m12s
   250K .......... .......... .......... .....

In [4]:
spark = SparkSession.builder.appName("RedfinDataAnalysis").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
redfin_data = spark.read.csv(
    "s3://store-raw-data-fs/city_market_tracker.tsv000.gz", 
    sep="\t", 
    header=True, 
    inferSchema=True
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
redfin_data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+---------------+-----------+--------------+--------+----------------------+----------------+------------+--------------+----------+--------------------+----------------+-----------------+---------------------+---------------------+-----------------+---------------------+---------------------+------------------+--------------------+--------------------+------------------+--------------------+--------------------+----------+-------------------+--------------------+-------------+--------------------+--------------------+------------+--------------------+--------------------+---------+--------------------+--------------------+----------------+--------------------+--------------------+----------+--------------+--------------+------------------+--------------------+--------------------+------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-----------------------+---------------------------+----------

In [11]:
redfin_data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- period_begin: date (nullable = true)
 |-- period_end: date (nullable = true)
 |-- period_duration: integer (nullable = true)
 |-- region_type: string (nullable = true)
 |-- region_type_id: integer (nullable = true)
 |-- table_id: integer (nullable = true)
 |-- is_seasonally_adjusted: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- property_type_id: integer (nullable = true)
 |-- median_sale_price: double (nullable = true)
 |-- median_sale_price_mom: double (nullable = true)
 |-- median_sale_price_yoy: double (nullable = true)
 |-- median_list_price: double (nullable = true)
 |-- median_list_price_mom: double (nullable = true)
 |-- median_list_price_yoy: double (nullable = true)
 |-- median_ppsf: double (nullable = true)
 |-- median_ppsf_mom: double (nullable = true)
 |-- median_ppsf_yoy: double (nul

In [12]:
redfin_data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['period_begin', 'period_end', 'period_duration', 'region_type', 'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region', 'city', 'state', 'state_code', 'property_type', 'property_type_id', 'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy', 'median_list_price', 'median_list_price_mom', 'median_list_price_yoy', 'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold', 'homes_sold_mom', 'homes_sold_yoy', 'pending_sales', 'pending_sales_mom', 'pending_sales_yoy', 'new_listings', 'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom', 'inventory_yoy', 'months_of_supply', 'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom', 'median_dom_mom', 'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom', 'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom', 'sold_above_list_yoy', 'price_drops', 'price_drops_mom', 'price_drops_yoy', 'off_market_in_two_wee

In [15]:
selected_columns = [
    'period_end',
    'period_duration',
    'city',
    'state',
    'property_type',
    'median_sale_price',
    'median_ppsf',
    'homes_sold',
    'inventory',
    'months_of_supply',
    'median_dom',
    'sold_above_list',
    'last_updated'
]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_redfin = redfin_data.select(selected_columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_redfin.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+------------+--------------+--------------------+-----------------+------------------+----------+---------+----------------+----------+------------------+-------------------+
|period_end|period_duration|        city|         state|       property_type|median_sale_price|       median_ppsf|homes_sold|inventory|months_of_supply|median_dom|   sold_above_list|       last_updated|
+----------+---------------+------------+--------------+--------------------+-----------------+------------------+----------+---------+----------------+----------+------------------+-------------------+
|2022-01-31|             30|    Sterling|      Virginia|     All Residential|         532000.0| 268.9473737177208|        28|       21|             0.8|        13|0.7857142857142857|2024-02-11 14:26:11|
|2013-03-31|             30|   Swansboro|North Carolina|Single Family Res...|         146250.0| 93.86333914559721|         4|       76|            19.0|       390|               0.0|2024-0

In [19]:
df_redfin.shape()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object has no attribute 'shape'
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1709833511192_0001/container_1709833511192_0001_01_000001/pyspark.zip/pyspark/sql/dataframe.py", line 3122, in __getattr__
    raise AttributeError(
AttributeError: 'DataFrame' object has no attribute 'shape'



In [20]:
num_rows = df_redfin.count()
num_columns = len(df_redfin.columns)

print("Number of rows: {}".format(num_rows))
print("Number of columns: {}".format(num_columns))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of rows: 5213526
Number of columns: 13

In [22]:
from pyspark.sql.functions import isnull

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
null_count = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
for i, col_name in enumerate(df_redfin,coumns):
    print(f'{col_name}: {null_count[i]}')

In [ ]:
df_redfin = df_redfin.na.drop()

In [ ]:
null_count = [df_redfin.where(isnull(col_name)).count() for col_name in df_redfin.columns]
null_count